In [1]:
import os
import mobilize_import as mi
import importlib
import pyodbc
import pyperclip
import pandas as pd
import datetime as dt
from time import sleep
from IPython.display import display, HTML
from getpass import getpass

importlib.reload(mi);
def dh(x): display(HTML(x))
def refresh(crsr): crsr.execute('select 0').fetchall()
dh("<style>.container { width:75% !important; }</style>")

def split_into_chunks(filename, limit=int(18*1024*1024)):
    filesize = os.path.getsize(filename)
    chunks = math.ceil(filesize / limit)
    chunksize= int(filesize / chunks)+1
    print(f"Splitting {filename} into {chunks} chunks.")
    with open(filename, 'rb') as fin:
        firstline = fin.readline()
        start = fin.tell()
        for i in range(chunks):
            sp = os.path.splitext(filename)
            filename_new = sp[0]+f'_{i+1}'+sp[1]
            with open(filename_new, 'wb') as fout:
                fout.write(firstline)
                offset = fin.seek( min(filesize,start+chunksize) ) - start
                if fin.tell() < filesize:
                    fin.readline()
                    offset = fin.tell() - start
                fin.seek(start)
                fout.write(fin.read(offset))
                print(f"Writing file {os.path.basename(filename_new)} of {offset/(2**20):.2f}MB.")
                start += offset


### Prep

#### Database connection
Connect to the database by running the following three cells, inputting your password when prompted.  If the final cell doesn't succeed, you can't connect to the database.

In [2]:
password=getpass("db password:")

db password:········


In [3]:
cnxn=None
cstring="Driver={Amazon Redshift (x64)}; " \
        "Server=redshift.zigzagcanal.orgs.civis.io; " \
        "Database=dev;" \
        "UID=tmcmanus;" \
        f"PWD={password}; " \
        "Port=5432" 

In [12]:
if cnxn: cnxn.close()
cnxn = pyodbc.connect(cstring, autocommit=True)
crsr = cnxn.cursor()

#### Choosing parameters

Replace `job` and `job_name` with the job ID and a job descriptor (prepended with `fld_` or `natl_`)  Set `csv_path` to the fully qualified directory where you intend to save the final CSV.

In [11]:
job=716
job_name='fld_nc_oo'
csv_path=r'C:\Users\mcman\OneDrive\Desktop\Work\Drive\Texting Lists\waterfall'
table_name=f'waterfall.t{dt.date.today():%Y%m%d}_{job}_{job_name}'
print(table_name)

waterfall.t20200218_716_fld_nc_oo


(Optional) Run the following cell to view all the requests in the job you chose.  Review `ak_title` and `combo` here.

In [10]:
sql = f"""
select job_id, request_id, mobilize_id, distance, capacity, earliest, latest, ak_title as title, combo, 'b-2020.com'||event1_link as link 
from waterfall.job_requests
where job_id in ({job})"""
pd.read_sql(sql,cnxn)

DatabaseError: Execution failed on sql '
select job_id, request_id, mobilize_id, distance, capacity, earliest, latest, ak_title as title, combo, 'b-2020.com'||event1_link as link 
from waterfall.job_requests
where job_id in (716)': ('00000', '[00000] [Amazon][Amazon Redshift] (30) Error occurred while trying to execute a query: [SQLState 00000] another command is already in progress\n (30) (SQLExecDirectW)')

### Automation
Run the following cells to execute the importer.

This cell creates the importer.  Add any exclusions to `exclusions`.  Change `implicit_exclusions` to `false` if you don't want to automatically include exclusions from lists for events that haven't happened yet in the same state.

In [7]:
importer = mi.create_importer(table_name=table_name,
                            job_id_in=job,
                            exclusions=[],
#                             title_override='Organize your Friends Party',
                            importer_type='watf',
#                             importer_type='bulk',

                            date_abs=[20200219,20200225]
                           )

# Uncomment and use this for non-waterfall pulls
# importer = mi.create_importer(table_name=table_name,
#                                 mob_id_in=ids,
#                                 job_id = job, #This is for job_tables
#                                 date_abs = ['20200107','20200111'],
#                                 exclusions=['waterfall.t20200104_EXCLUSION_SD_OOS',
#                                             'waterfall.t20200104_EXCLUSION_kc_OOS',
#                                             'waterfall.t20200104_EXCLUSION_linc_OOS'],
#                                 distance=40,
#                                 capacity=60,
#                                 importer_type='base'
#                            )

# Run this to copy the query to your clipboard
# importer.full_query(True); 

implicit_exclusions= True


This cell runs through the verify step.

In [8]:
crsr = cnxn.cursor() 
crsr.execute(importer.input)
crsr.execute(importer.events)
refresh(crsr)
# df = pd.read_sql('select job_id, request_id, event_id, mobilize_id, event_category, event_title, event_state, event_date, capacity, event_distance_override, combo from events', cnxn)
df = pd.read_sql('select * from events', cnxn)
if len(df) == 0:
    raise Exception("No events found")
else:
    print("EVENTS:")
    dh(df.to_html(index=False))
    dh('<hr>')
    

if(implicit_exclusions):
    states = '\',\''.join(list(df.event_state.drop_duplicates())+['NATL'])
    sql = f"select distinct table_name from waterfall.job_tables where state in ('{states}') and event_date > getdate() and table_name <> '{table_name}'" 
    df_excl = pd.read_sql(sql, cnxn)
    excl = df_excl['table_name']
    importer.exclusions += list(excl)
    importer.make_exclusions()
    if not excl.empty:
        print("EXCLUSIONS:")
        print('\n'.join(importer.exclusions))
        dh('<hr>')
    

    
val = crsr.execute(importer.event_proximity).execute('select count(*) from event_proximity').fetchval();
crsr.fetchall()
if val:
    print("Event_proximity count: ",val)
    dh('<hr>')
else:
    raise Exception("event_proximity empty")
    
crsr.execute(importer.user_assignment)
refresh(crsr)
df2 = pd.read_sql(importer.verify, cnxn)
print("VERIFY:")
dh(df2.to_html(index=False))
dh('<hr>')

KeyboardInterrupt: 

This cell creates the final table, with the table name you created at the start.

In [9]:
# crsr.execute(f"update {table_name} set event_title1='Debate Watch Party' where event_campaign1 ilike '%debate%'")

In [90]:
fcount = crsr.execute(importer.final_table).execute(importer.job_tables).execute(f"select count(*) from {table_name}").fetchval()
crsr.fetchall()
print("Final table size:",fcount)

Final table size: 165662


This cell downloads the table and saves it to a CSV.  Not tested for large tables.

### Exporting 

In [91]:
schema, no_schema = table_name.split('.') 
final_df = pd.read_sql(f"select * from {table_name}", cnxn)

This cell exports to a csv

In [92]:
pyperclip.copy(table_name)
print(table_name)

waterfall.t20200205_492_fld_ca_canvasses


In [93]:
print(final_df['day1'].drop_duplicates())
print(final_df['event_title1'].drop_duplicates())

0           Sunday, Feb 9
56        Saturday, Feb 8
51572    Thursday, Feb 13
Name: day1, dtype: object
0    Canvass
Name: event_title1, dtype: object


In [84]:
final_df['event_title1'] = final_df['event_title1'].apply(lambda x: 'Canvass')

In [94]:
final_df.to_csv(os.path.join(csv_path, no_schema+'.csv'))

#### The following cells let you split out the result by city

In [257]:
cities = list(final_df['event1_city'].drop_duplicates())
for city in cities:
    final_df.loc[final_df['event1_city'] == city].to_csv(os.path.join(csv_path, no_schema+f'_{city}.csv'),index=False)

#### The following cells show you how to split out the result csv by `event1_date`

In [45]:
dates = final_df['day1'].drop_duplicates()
dl = sorted(list(dates), key=lambda x: x[-2:])
for i, d in enumerate(dl):
    print(i, d)

0 Thursday, Feb 6
1 Friday, Feb 7
2 Saturday, Feb 8
3 Sunday, Feb 9
4 Monday, Feb 10
5 Tuesday, Feb 11
6 Wednesday, Feb 12
7 Thursday, Feb 13
8 Friday, Feb 14


In [46]:
strides = [0,4, len(dl)]

In [47]:
for i in range(1,len(strides)):
    s = (strides[i-1], strides[i])
    print(f"Exporting events from {dl[s[0]]} through {dl[s[1]-1]} (inclusive).")
    fname = os.path.join(csv_path, no_schema+f'_{i}.csv')
    print(fname, end='... ')
    final_df.loc[final_df['day1'].isin(dl[s[0]:s[1]])].to_csv(fname)
    print("Success!")

Exporting events from Thursday, Feb 6 through Sunday, Feb 9 (inclusive).
C:\Users\mcman\OneDrive\Desktop\Work\Drive\Texting Lists\waterfall\t20200205_521_522_fld_volr_phonebanks_1.csv... Success!
Exporting events from Monday, Feb 10 through Friday, Feb 14 (inclusive).
C:\Users\mcman\OneDrive\Desktop\Work\Drive\Texting Lists\waterfall\t20200205_521_522_fld_volr_phonebanks_2.csv... Success!
